In [22]:
import pandas
import pickle
import numpy as np

merged_data = pandas.read_pickle('../data/data-preperation-output.pickle')
merged_data.head()

,code_module,code_presentation,id_student,assessments,vles,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,date_registration,date_unregistration,module_presentation_length
0,AAA,2013J,11391,"[[1752, 18, 78.0, TMA, 19.0, 10.0], [1753, 53,...","[[546614, -5, homepage, 7], [546614, 0, homepa...",M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,-159.0,NaN,268
1,AAA,2013J,28400,"[[1752, 22, 70.0, TMA, 19.0, 10.0], [1753, 52,...","[[546614, -10, homepage, 11], [546614, -8, hom...",F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,-53.0,NaN,268
2,AAA,2013J,31604,"[[1752, 17, 72.0, TMA, 19.0, 10.0], [1753, 51,...","[[546614, -10, homepage, 11], [546614, -9, hom...",F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,-52.0,NaN,268
3,AAA,2013J,32885,"[[1752, 26, 69.0, TMA, 19.0, 10.0], [1753, 75,...","[[546614, -10, homepage, 13], [546614, -9, hom...",F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,-176.0,NaN,268
4,AAA,2013J,38053,"[[1752, 19, 79.0, TMA, 19.0, 10.0], [1753, 64,...","[[546614, -10, homepage, 13], [546614, -9, hom...",M,Wales,A Level or Equivalent,80-90%,35-55,0,60,N,Pass,-110.0,NaN,268


In [23]:
def calculate_amount_assessments(row):
    row['assessments_amount'] = len(row['assessments'])
    return row

merged_data = merged_data.apply(calculate_amount_assessments, axis=1)
merged_data['assessments_amount'].describe()

count    25793.000000
mean         6.736091
std          3.770717
min          1.000000
25%          4.000000
50%          7.000000
75%         10.000000
max         14.000000
Name: assessments_amount, dtype: float64

In [24]:
import math

# analyze the max weight of a course
assessment_lists_per_student_per_course = merged_data.groupby(['code_module', 'code_presentation']).agg({
    "assessments": lambda x: list(x),
}).reset_index()

def analyze_max_weight_per_course_per_presentation(row):
    asessments_of_students = row['assessments']
    max_weight = 0
    for asessments_of_student in asessments_of_students:
        weights = [weight for (id_assessment, date_submitted, score, assessment_type, date, weight) in asessments_of_student]
        student_max_weight = sum(weights)
        if student_max_weight > max_weight:
            max_weight = student_max_weight
        

    row['max_weight'] = max_weight
    return row

max_weights_per_course_per_presentation = assessment_lists_per_student_per_course.apply(analyze_max_weight_per_course_per_presentation, axis = 1)
max_weights_per_course_per_presentation = max_weights_per_course_per_presentation.drop('assessments', axis=1).set_index(['code_module', 'code_presentation'])
max_weights_per_course_per_presentation.head(21)

max_weight
code_module code_presentation            
AAA         2013J                   100.0
            2014J                   100.0
BBB         2013B                   100.0
            2013J                   100.0
            2014B                   100.0
            2014J                   100.0
CCC         2014B                   200.0
            2014J                   200.0
DDD         2013B                   200.0
            2013J                   200.0
            2014B                   200.0
            2014J                   200.0
EEE         2013J                   100.0
            2014B                   100.0
            2014J                   100.0
FFF         2013B                   100.0
            2013J                   100.0
            2014B                   100.0
            2014J                   100.0
GGG         2013J                     0.0
            2014B                     0.0

In [25]:
def analyze_average_grade(assessments):
    total_weight = 0
    total_score = 0

    for assessment in assessments:
        id_assessment, date_submitted, score, assessment_type, date, weight = assessment
        if (not(type(score) == float)):
            print(score)
        total_weight +=  weight
        total_score += score * weight


    if total_weight > 0:
        average_score = total_score / total_weight
        return average_score
    else:
        # sometimes there are assessments, except there are no weights
        if (len(assessments) > 0):
            scores = [assessment[2] for assessment in assessments]
            return np.mean(scores)
            
        return math.nan

merged_data['average_score'] = merged_data['assessments'].transform(analyze_average_grade)
merged_data.head()

,code_module,code_presentation,id_student,assessments,vles,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,date_registration,date_unregistration,module_presentation_length,assessments_amount,average_score
0,AAA,2013J,11391,"[[1752, 18, 78.0, TMA, 19.0, 10.0], [1753, 53,...","[[546614, -5, homepage, 7], [546614, 0, homepa...",M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,-159.0,NaN,268,5,82.4
1,AAA,2013J,28400,"[[1752, 22, 70.0, TMA, 19.0, 10.0], [1753, 52,...","[[546614, -10, homepage, 11], [546614, -8, hom...",F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,-53.0,NaN,268,5,65.4
2,AAA,2013J,31604,"[[1752, 17, 72.0, TMA, 19.0, 10.0], [1753, 51,...","[[546614, -10, homepage, 11], [546614, -9, hom...",F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,-52.0,NaN,268,5,76.3
3,AAA,2013J,32885,"[[1752, 26, 69.0, TMA, 19.0, 10.0], [1753, 75,...","[[546614, -10, homepage, 13], [546614, -9, hom...",F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,-176.0,NaN,268,5,55.0
4,AAA,2013J,38053,"[[1752, 19, 79.0, TMA, 19.0, 10.0], [1753, 64,...","[[546614, -10, homepage, 13], [546614, -9, hom...",M,Wales,A Level or Equivalent,80-90%,35-55,0,60,N,Pass,-110.0,NaN,268,5,66.9


In [26]:
# jort
# sum of all clicks

def sum_clicks_vles(vles):
    sum_clicks = 0
    for [id_site, date, activity_type, sum_click] in vles:
        sum_clicks += sum_click
    return sum_clicks

merged_data["sum_clicks"] = merged_data["vles"].transform(sum_clicks_vles)
merged_data.head()

,code_module,code_presentation,id_student,assessments,vles,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,date_registration,date_unregistration,module_presentation_length,assessments_amount,average_score,sum_clicks
0,AAA,2013J,11391,"[[1752, 18, 78.0, TMA, 19.0, 10.0], [1753, 53,...","[[546614, -5, homepage, 7], [546614, 0, homepa...",M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,-159.0,NaN,268,5,82.4,934
1,AAA,2013J,28400,"[[1752, 22, 70.0, TMA, 19.0, 10.0], [1753, 52,...","[[546614, -10, homepage, 11], [546614, -8, hom...",F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,-53.0,NaN,268,5,65.4,1435
2,AAA,2013J,31604,"[[1752, 17, 72.0, TMA, 19.0, 10.0], [1753, 51,...","[[546614, -10, homepage, 11], [546614, -9, hom...",F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,-52.0,NaN,268,5,76.3,2158
3,AAA,2013J,32885,"[[1752, 26, 69.0, TMA, 19.0, 10.0], [1753, 75,...","[[546614, -10, homepage, 13], [546614, -9, hom...",F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,-176.0,NaN,268,5,55.0,1034
4,AAA,2013J,38053,"[[1752, 19, 79.0, TMA, 19.0, 10.0], [1753, 64,...","[[546614, -10, homepage, 13], [546614, -9, hom...",M,Wales,A Level or Equivalent,80-90%,35-55,0,60,N,Pass,-110.0,NaN,268,5,66.9,2445


In [27]:
# calculate course difficulty
import statistics

average_scores_per_course = merged_data.groupby(['code_module', 'code_presentation']).agg({
    "average_score": lambda x: list(x),
}).reset_index()

def calculate_course_average_score(row):
    average_scores = np.array(row['average_score'])
    average_score_without_nan = average_scores[~np.isnan(average_scores)]
    if (len(average_score_without_nan) == 0):
        row['course_difficulty'] = np.NaN
    else:
        row['course_difficulty'] = np.mean(average_score_without_nan)
    return row
    
average_scores_per_course = average_scores_per_course.apply(calculate_course_average_score, axis=1)
average_scores_per_course = average_scores_per_course.drop(['average_score'], axis=1)
average_scores_per_course

,code_module,code_presentation,course_difficulty
0,AAA,2013J,68.432979
1,AAA,2014J,67.741056
2,BBB,2013B,70.885545
3,BBB,2013J,70.831964
4,BBB,2014B,71.575226
5,BBB,2014J,65.722549
6,CCC,2014B,61.591866
7,CCC,2014J,66.713463
8,DDD,2013B,60.755244
9,DDD,2013J,64.730415


In [28]:
# merge difficulty course with the data
merged_data = pandas.merge(merged_data, average_scores_per_course, on=['code_module', 'code_presentation'])
merged_data.head()

,code_module,code_presentation,id_student,assessments,vles,gender,region,highest_education,imd_band,age_band,...,studied_credits,disability,final_result,date_registration,date_unregistration,module_presentation_length,assessments_amount,average_score,sum_clicks,course_difficulty
0,AAA,2013J,11391,"[[1752, 18, 78.0, TMA, 19.0, 10.0], [1753, 53,...","[[546614, -5, homepage, 7], [546614, 0, homepa...",M,East Anglian Region,HE Qualification,90-100%,55<=,...,240,N,Pass,-159.0,NaN,268,5,82.4,934,68.432979
1,AAA,2013J,28400,"[[1752, 22, 70.0, TMA, 19.0, 10.0], [1753, 52,...","[[546614, -10, homepage, 11], [546614, -8, hom...",F,Scotland,HE Qualification,20-30%,35-55,...,60,N,Pass,-53.0,NaN,268,5,65.4,1435,68.432979
2,AAA,2013J,31604,"[[1752, 17, 72.0, TMA, 19.0, 10.0], [1753, 51,...","[[546614, -10, homepage, 11], [546614, -9, hom...",F,South East Region,A Level or Equivalent,50-60%,35-55,...,60,N,Pass,-52.0,NaN,268,5,76.3,2158,68.432979
3,AAA,2013J,32885,"[[1752, 26, 69.0, TMA, 19.0, 10.0], [1753, 75,...","[[546614, -10, homepage, 13], [546614, -9, hom...",F,West Midlands Region,Lower Than A Level,50-60%,0-35,...,60,N,Pass,-176.0,NaN,268,5,55.0,1034,68.432979
4,AAA,2013J,38053,"[[1752, 19, 79.0, TMA, 19.0, 10.0], [1753, 64,...","[[546614, -10, homepage, 13], [546614, -9, hom...",M,Wales,A Level or Equivalent,80-90%,35-55,...,60,N,Pass,-110.0,NaN,268,5,66.9,2445,68.432979


In [29]:
import itertools

assessment_columns = ['id_assessment', 'date_submitted', 'score', 'assessment_type', 'date', 'weight']
def create_columns_for_assessments(row):
    assessments = row['assessments']
    amount_of_assessments_to_put_in_data = 7
    assessments_to_process = assessments[:amount_of_assessments_to_put_in_data]
    for index, assessment in enumerate(assessments_to_process, start=1):
        id_assessment, date_submitted, score, assessment_type, date, weight = assessment
        assessment_score_column_name = f'assessment_{index}_score'
        row[assessment_score_column_name] = score
        # assessment_weight_column_name = f'assessment_{index}_weight'
        # row[assessment_weight_column_name] = weight

    return row
    
merged_data = merged_data.apply(create_columns_for_assessments, axis=1)
merged_data.head()

,age_band,assessment_1_score,assessment_2_score,assessment_3_score,assessment_4_score,assessment_5_score,assessment_6_score,assessment_7_score,assessments,assessments_amount,...,gender,highest_education,id_student,imd_band,module_presentation_length,num_of_prev_attempts,region,studied_credits,sum_clicks,vles
0,55<=,78.0,85.0,80.0,85.0,82.0,NaN,NaN,"[[1752, 18, 78.0, TMA, 19.0, 10.0], [1753, 53,...",5,...,M,HE Qualification,11391,90-100%,268,0,East Anglian Region,240,934,"[[546614, -5, homepage, 7], [546614, 0, homepa..."
1,35-55,70.0,68.0,70.0,64.0,60.0,NaN,NaN,"[[1752, 22, 70.0, TMA, 19.0, 10.0], [1753, 52,...",5,...,F,HE Qualification,28400,20-30%,268,0,Scotland,60,1435,"[[546614, -10, homepage, 11], [546614, -8, hom..."
2,35-55,72.0,71.0,74.0,88.0,75.0,NaN,NaN,"[[1752, 17, 72.0, TMA, 19.0, 10.0], [1753, 51,...",5,...,F,A Level or Equivalent,31604,50-60%,268,0,South East Region,60,2158,"[[546614, -10, homepage, 11], [546614, -9, hom..."
3,0-35,69.0,30.0,63.0,35.0,75.0,NaN,NaN,"[[1752, 26, 69.0, TMA, 19.0, 10.0], [1753, 75,...",5,...,F,Lower Than A Level,32885,50-60%,268,0,West Midlands Region,60,1034,"[[546614, -10, homepage, 13], [546614, -9, hom..."
4,35-55,79.0,69.0,74.0,50.0,68.0,NaN,NaN,"[[1752, 19, 79.0, TMA, 19.0, 10.0], [1753, 64,...",5,...,M,A Level or Equivalent,38053,80-90%,268,0,Wales,60,2445,"[[546614, -10, homepage, 13], [546614, -9, hom..."


In [30]:
vle_columns = ['id_site', 'date', 'activity_type', 'sum_click']
def create_columns_for_clicks_per_month (row):
    vles = row['vles']
    vles_df = pandas.DataFrame(vles, columns=vle_columns)
    group_by_amount_of_days = 30
    column_name = 'month_number'
    vles_df[column_name] = (np.floor(np.clip(vles_df['date'] / group_by_amount_of_days, 0, None)) + 1).astype(int)
    clicks_per_week = vles_df.groupby([column_name])['sum_click'].agg('sum')

    for month_number in range(1, 8):
        if month_number in clicks_per_week.index:
            row[f'vle_click_month_{month_number}'] = clicks_per_week[month_number]
        else:
            row[f'vle_click_month_{month_number}'] = 0

    return row
    
merged_data = merged_data.apply(create_columns_for_clicks_per_month , axis=1)
merged_data.head()

,age_band,assessment_1_score,assessment_2_score,assessment_3_score,assessment_4_score,assessment_5_score,assessment_6_score,assessment_7_score,assessments,assessments_amount,...,studied_credits,sum_clicks,vles,vle_click_month_1,vle_click_month_2,vle_click_month_3,vle_click_month_4,vle_click_month_5,vle_click_month_6,vle_click_month_7
0,55<=,78.0,85.0,80.0,85.0,82.0,NaN,NaN,"[[1752, 18, 78.0, TMA, 19.0, 10.0], [1753, 53,...",5,...,240,934,"[[546614, -5, homepage, 7], [546614, 0, homepa...",419,110,58,41,82,3,35
1,35-55,70.0,68.0,70.0,64.0,60.0,NaN,NaN,"[[1752, 22, 70.0, TMA, 19.0, 10.0], [1753, 52,...",5,...,60,1435,"[[546614, -10, homepage, 11], [546614, -8, hom...",618,51,148,79,111,114,208
2,35-55,72.0,71.0,74.0,88.0,75.0,NaN,NaN,"[[1752, 17, 72.0, TMA, 19.0, 10.0], [1753, 51,...",5,...,60,2158,"[[546614, -10, homepage, 11], [546614, -9, hom...",503,301,184,269,202,257,149
3,0-35,69.0,30.0,63.0,35.0,75.0,NaN,NaN,"[[1752, 26, 69.0, TMA, 19.0, 10.0], [1753, 75,...",5,...,60,1034,"[[546614, -10, homepage, 13], [546614, -9, hom...",567,35,81,64,78,13,75
4,35-55,79.0,69.0,74.0,50.0,68.0,NaN,NaN,"[[1752, 19, 79.0, TMA, 19.0, 10.0], [1753, 64,...",5,...,60,2445,"[[546614, -10, homepage, 13], [546614, -9, hom...",727,277,56,215,110,162,225


In [31]:
merged_data.loc[0]

age_band                                                                   55<=
assessment_1_score                                                         78.0
assessment_2_score                                                         85.0
assessment_3_score                                                         80.0
assessment_4_score                                                         85.0
assessment_5_score                                                         82.0
assessment_6_score                                                          NaN
assessment_7_score                                                          NaN
assessments                   [[1752, 18, 78.0, TMA, 19.0, 10.0], [1753, 53,...
assessments_amount                                                            5
average_score                                                              82.4
code_module                                                                 AAA
code_presentation                       

In [32]:

final_data = merged_data.drop(
    [
        "assessments", 
        "vles",
    ], 
    axis=1
)
final_data.to_pickle("../data/data-extension-output.pickle")

# assesments per month - done
# clicks per month - done
# todo filter out average score
# remove code module
# remove presentation year
# remove student number